# Forecasting
This notebook is meant to show how the current forecaster works (data, architecture, loss)

In [79]:
from pathlib import Path

import sys
sys.path.append("..")

from src.models import Model

## Load in data (used for Kenya model)

In [5]:
m = Model.load_from_checkpoint("../data/models/Kenya.ckpt")

In [53]:
train, val, test = [m.get_dataset(subset).x for subset in ["training", "validation", "testing"]]

100%|██████████| 8241/8241 [00:00<00:00, 8708.55it/s]
Directory: ../data/features/geowiki_landcover_2017/testing not found. Use command: `dvc pull` to get the latest data.
100%|██████████| 884/884 [00:00<00:00, 2671.95it/s]


In [55]:
# (examples, timesteps, bands)
train.shape

torch.Size([37943, 12, 12])

## Create forecasting dataset

In [61]:
forecast_timesteps = 7
input_timesteps = 12 - forecast_timesteps

def create_X_Y(dataset):
    X = dataset[:, :input_timesteps, :]
    Y = dataset[:, input_timesteps:, :]
    return X, Y

In [64]:
X_train, Y_train = create_X_Y(train)
X_val, Y_val = create_X_Y(val)
X_test, Y_test = create_X_Y(test)

## Info about the forecaster

In [88]:
# How it looks
m.forecaster

Forecaster(
  (lstm): UnrolledLSTM(
    (rnn): UnrolledLSTMCell(
      (forget_gate): Sequential(
        (0): Linear(in_features=268, out_features=256, bias=True)
        (1): Sigmoid()
      )
      (update_gate): Sequential(
        (0): Linear(in_features=268, out_features=256, bias=True)
        (1): Sigmoid()
      )
      (update_candidates): Sequential(
        (0): Linear(in_features=268, out_features=256, bias=True)
        (1): Tanh()
      )
      (output_gate): Sequential(
        (0): Linear(in_features=268, out_features=256, bias=True)
        (1): Sigmoid()
      )
      (cell_state_activation): Tanh()
    )
    (dropout): VariationalDropout()
  )
  (to_bands): Linear(in_features=256, out_features=12, bias=True)
)

## Use forecaster (from Kenya model) to make predictions and computer loss

In [86]:
def predict(X):
    Y_pred_unprocessed = m.forecaster(X) # predicts every month except first
    Y_pred = Y_pred_unprocessed[:, -forecast_timesteps:, :] # We only need last 7
    return Y_pred

def compute_loss(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return m.forecaster_loss(Y, Y_pred)

In [84]:
Y_train_pred = predict(X_train)
compute_loss(Y_train, Y_train_pred)

tensor(0.2254, grad_fn=<MeanBackward0>)

In [87]:
Y_val_pred = predict(X_val)
compute_loss(Y_val, Y_val_pred)

tensor(0.2446, grad_fn=<MeanBackward0>)